# 基于LlamaIndex构建RAG应用

## 1. 简介
本文主要介绍如何在LlamaIndex中使用百炼提供的检索增强服务。

### 1.1 功能依赖
暂无

### 1.2 前期准备
* 已开通百炼服务：[开通阿里云百炼大模型服务产品](https://help.aliyun.com/document_detail/2586399.html?spm=a2c4g.2784257.0.i6)。
* 已创建和获取API_KEY: [获取API-KEY](https://help.aliyun.com/document_detail/2712195.html)。

## 2. 代码示例

### 2.1 安装LlamaIndex依赖包

In [ ]:
!pip install llama-index-core
!pip install llama-index-llms-dashscope
!pip install llama-index-indices-managed-dashscope

### 2.2 文件解析
准备您的知识库文件，可以将所有文件放在一个文件夹中也可以是多个独立的文件。目前支持的文件格式是docx、doc、pdf。准备好之后，使用DashScopeParse进行解析。

In [ ]:
import os

from llama_index.readers.dashscope.base import DashScopeParse
from llama_index.readers.dashscope.utils import ResultType

os.environ['DASHSCOPE_API_KEY'] = "<Your API Key>"
os.environ['DASHSCOPE_WORKSPACE_ID'] = "<Your Workspace id, Default workspace is empty.>"

# 多个独立文件
file = [
    # 需要解析的文件，支持pdf,doc,docx
]
parse = DashScopeParse(result_type=ResultType.DASHSCOPE_DOCMIND)
documents = parse.load_data(file_path=file)

# 使用SimpleDirectoryReader读取文件夹下所有文件
#from llama_index.core import SimpleDirectoryReader
#parse = DashScopeParse(result_type=ResultType.DASHSCOPE_DOCMIND)
#file_extractor = {".pdf": parse, '.doc': parse, '.docx': parse}
#documents = SimpleDirectoryReader(
#    "your_folder", file_extractor=file_extractor
#).load_data(num_workers=1)

### 2.3 创建索引
通过上一步文档解析得到的documents创建知识库索引

In [ ]:
from llama_index.indices.managed.dashscope import DashScopeCloudIndex

# create a new index
index = DashScopeCloudIndex.from_documents(
    documents,
    "my_first_index",
    verbose=True,
)

### 2.4 读取索引
您可以使用知识库名称在LlamaIndex中初始化您创建好的知识索引



In [ ]:
my_index = DashScopeCloudIndex("my_first_index")


### 2.5 获得retriever
您可以从index对象中快速获得您的retriever，或者使用知识库名称初始化您的DashScopeCloudRetriever



In [ ]:
# convert from index
retriever = index.as_retriever()

# or initialize from DashScopeCloudRetriever
# from llama_index.indices.managed.dashscope.retriever import DashScopeCloudRetriever
# retriever = DashScopeCloudRetriever("my_first_index")

nodes = retriever.retrieve("my query")

### 2.6 获得query engine

In [ ]:
from llama_index.llms.dashscope import DashScope, DashScopeGenerationModels

dashscope_llm = DashScope(
  model_name=DashScopeGenerationModels.QWEN_MAX, api_key=os.environ["DASHSCOPE_API_KEY"]
)
query_engine = index.as_query_engine(llm=dashscope_llm)

### 2.7 从索引新增/删除文档

In [ ]:
# add documents to index
index.insert(documents)

# delete documents from index
doc_id = "doc_123"
index.delete_ref_doc([doc_id])